In [15]:
from pathlib import Path
from datetime import date
import polars as pl

csv_path = Path("data/Stock level signals.csv")

stock_data = pl.read_csv(csv_path)
stock_data = stock_data.with_columns(
    pl.col("date").str.to_date("%m/%Y").dt.replace(day=1).dt.offset_by("1mo").dt.offset_by("-1d")
)
# stock_data.filter(pl.col("permno") == 10006)
signals_permno = stock_data.select("permno","date").unique()
signals_permno = signals_permno.sort("permno")

signals_permno

permno,date
i64,date
10006,1971-07-31
10006,1979-11-30
10006,1978-08-31
10006,1968-03-31
10006,1965-05-31
…,…
93436,2018-07-31
93436,2018-05-31
93436,2019-05-31


In [16]:
stock_data.null_count()

permno,date,re,size,value,prof,dur,valprof,fscore,debtiss,repurch,nissa,accruals,growth,aturnover,gmargins,divp,ep,cfp,noa,inv,invcap,igrowth,sgrowth,lev,roaa,roea,sp,gltnoa,divg,invaci,mom,indmom,valmom,valmomprof,shortint,mom12,momrev,lrrev,valuem,nissm,sue,roe,rome,roa,strev,ivol,betaarb,season,indrrev,indrrevlv,indmomrev,ciss,price,age,shvol,ipo
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,236,11472,76177,69981,266383,184672,0,103910,103910,34634,200760,88033,64324,64543,234980,64556,128597,192980,173582,139657,164457,89901,63068,62941,75755,64247,285706,249420,672841,19174,3735,184486,188981,302432,23285,37652,118840,63007,24325,505599,183234,173265,226245,0,7935,72712,116168,3730,291480,291482,119069,0,0,28473,684662


In [17]:
import pandas as pd
import numpy as np

pd_df = pd.read_csv(
    "data/F-F_Research_Data_5_Factors_2x3.csv",
    header=None,
    skiprows=4,
    nrows=746,
    sep=r"\s+",
    engine="python",
    names=["date", "Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"],
)

for col in pd_df.columns:
    pd_df[col] = pd_df[col].astype(str).str.rstrip(',')

for col in ["Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"]:
    pd_df[col] = pd_df[col].astype(float)

print("Pandas shape:", pd_df.shape)

ff_data = pl.from_pandas(pd_df)

ff_data = ff_data.with_columns([
    (pl.col("Mkt-RF") / 100).alias("Mkt-RF"),
    (pl.col("SMB") / 100).alias("SMB"),
    (pl.col("HML") / 100).alias("HML"),
    (pl.col("RMW") / 100).alias("RMW"),
    (pl.col("CMA") / 100).alias("CMA"),
    (pl.col("RF") / 100).alias("RF"),
])

ff_data = ff_data.with_columns(
    pl.col("date").str.to_date("%Y%m").dt.replace(day=1).dt.offset_by("1mo").dt.offset_by("-1d").alias("parsed_date"),
)
ff_data = ff_data.drop("date").rename({"parsed_date": "date"})
print("Polars shape:", ff_data.shape)
ff_data

Pandas shape: (746, 7)
Polars shape: (746, 7)


Mkt-RF,SMB,HML,RMW,CMA,RF,date
f64,f64,f64,f64,f64,f64,date
-0.0039,-0.0048,-0.0081,0.0064,-0.0115,0.0027,1963-07-31
0.0508,-0.008,0.017,0.004,-0.0038,0.0025,1963-08-31
-0.0157,-0.0043,0.0,-0.0078,0.0015,0.0027,1963-09-30
0.0254,-0.0134,-0.0004,0.0279,-0.0225,0.0029,1963-10-31
-0.0086,-0.0085,0.0173,-0.0043,0.0227,0.0027,1963-11-30
…,…,…,…,…,…,…
-0.0084,-0.0186,-0.034,-0.0285,-0.0267,0.0035,2025-04-30
0.0606,-0.0072,-0.0288,0.0126,0.0251,0.0038,2025-05-31
0.0486,-0.0002,-0.016,-0.0319,0.0145,0.0034,2025-06-30


In [26]:
CRSP_data= pl.read_csv("../OneDrive_1_12-11-2025/msf_raw_1964To2023.csv",dtypes={
    "CUSIP": pl.Utf8, 
    "RET": pl.Utf8, 
    "RETX": pl.Utf8, 
    "SICCD": pl.Utf8,
    # "Date": pl.Date,
})

CRSP_data = CRSP_data.with_columns(
    pl.when(pl.col("RET").is_in(["B", "C", ".", ""]))
      .then(None)
      .otherwise(pl.col("RET"))
      .cast(pl.Float64)
      .alias("RET"),
    pl.when(pl.col("RETX").is_in(["B", "C", ".", ""]))
      .then(None)
      .otherwise(pl.col("RETX"))
      .cast(pl.Float64)
      .alias("RETX"),
    pl.when(pl.col("SICCD").is_in(["Z", ".", ""]))
      .then(None)
      .otherwise(pl.col("SICCD"))
      .cast(pl.Int64)
      .alias("SICCD"),
)

CRSP_data= CRSP_data['DATE','PERMNO','SICCD','CUSIP','BIDLO','ASKHI','PRC','VOL','RET','BID','ASK','SHROUT','RETX','vwretd']


df_with_parsed_dates = CRSP_data.with_columns(
    pl.col("DATE").cast(pl.Utf8).str.to_date("%Y%m%d").alias("parsed_date"),
)

df_with_parsed_dates = df_with_parsed_dates.sort(["PERMNO", "parsed_date"]).with_columns(
    pl.col("PRC").forward_fill().over("PERMNO")
)

df_with_parsed_dates = df_with_parsed_dates.with_columns(
    (pl.col("PRC") * pl.col("SHROUT")).alias("mkt_cap_monthly")
)
df_with_parsed_dates = df_with_parsed_dates.drop("DATE").rename({"parsed_date":"date"})
df_with_parsed_dates = df_with_parsed_dates.with_columns(
    pl.col("date").dt.replace(day=1).dt.offset_by("1mo").dt.offset_by("-1d").alias("parsed_date"),
)
df_with_parsed_dates = df_with_parsed_dates.drop("date").rename({"parsed_date": "date"})


permno_industry_map = df_with_parsed_dates.select([
    pl.col("PERMNO"), pl.col("date"), pl.col("mkt_cap_monthly"), pl.col("SHROUT")]).unique()
permno_industry_map= permno_industry_map.sort("PERMNO")
permno_crsp_data = permno_industry_map.rename({"PERMNO":"permno"})
permno_crsp_data.sort(["date"])

/var/folders/5h/7gyshb790f549lhn9zp_6x380000gn/T/ipykernel_43490/239963421.py:1: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  CRSP_data= pl.read_csv("../OneDrive_1_12-11-2025/msf_raw_1964To2023.csv",dtypes={


permno,date,mkt_cap_monthly,SHROUT
i64,date,f64,i64
10057,1964-01-31,31250.0,500
10241,1964-01-31,671498.0,16378
10575,1964-01-31,693747.0,24342
10591,1964-01-31,170681.0,2636
10647,1964-01-31,5310.625,293
…,…,…,…
93423,2023-12-31,2.0951e6,83537
93426,2023-12-31,426215.7,12510
93427,2023-12-31,6.9149e6,36331


In [27]:
signals_permno.filter(pl.col("date").is_between(date(1964, 2, 29), date(1964, 3, 1)))

permno,date
i64,date
10006,1964-02-29
10030,1964-02-29
10102,1964-02-29
10137,1964-02-29
10145,1964-02-29
…,…
53023,1964-02-29
58843,1964-02-29
68195,1964-02-29


In [28]:
permno_crsp_ff_data = permno_crsp_data.join(ff_data.select(["date","Mkt-RF"]), on="date", how="left")
permno_crsp_ff_data.sort("date")

permno,date,mkt_cap_monthly,SHROUT,Mkt-RF
i64,date,f64,i64,f64
10057,1964-01-31,31250.0,500,0.0227
10241,1964-01-31,671498.0,16378,0.0227
10575,1964-01-31,693747.0,24342,0.0227
10591,1964-01-31,170681.0,2636,0.0227
10647,1964-01-31,5310.625,293,0.0227
…,…,…,…,…
93423,2023-12-31,2.0951e6,83537,0.0486
93426,2023-12-31,426215.7,12510,0.0486
93427,2023-12-31,6.9149e6,36331,0.0486


In [29]:
signals_permno_crsp_ff = signals_permno.join(permno_crsp_ff_data, on=["permno", "date"], how="inner")
signals_permno_crsp_ff.sort("date")

permno,date,mkt_cap_monthly,SHROUT,Mkt-RF
i64,date,f64,i64,f64
10241,1964-01-31,671498.0,16378,0.0227
10575,1964-01-31,693747.0,24342,0.0227
10591,1964-01-31,170681.0,2636,0.0227
10671,1964-01-31,59626.0,4259,0.0227
11260,1964-01-31,1.405409e6,36152,0.0227
…,…,…,…,…
93312,2019-12-31,1.563244e7,254600,0.0276
93330,2019-12-31,5.4000e6,41360,0.0276
93374,2019-12-31,6.5514e6,112336,0.0276


In [30]:
stock_data_with_crsp_ff = stock_data.join(
    signals_permno_crsp_ff, 
    on=["permno", "date"], 
    how="inner"
)
stock_data_with_crsp_ff.sort("date")

permno,date,re,size,value,prof,dur,valprof,fscore,debtiss,repurch,nissa,accruals,growth,aturnover,gmargins,divp,ep,cfp,noa,inv,invcap,igrowth,sgrowth,lev,roaa,roea,sp,gltnoa,divg,invaci,mom,indmom,valmom,valmomprof,shortint,mom12,momrev,lrrev,valuem,nissm,sue,roe,rome,roa,strev,ivol,betaarb,season,indrrev,indrrevlv,indmomrev,ciss,price,age,shvol,ipo,mkt_cap_monthly,SHROUT,Mkt-RF
i64,date,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,str,str,str,str,f64,str,f64,f64,f64,str,str,f64,f64,f64,f64,str,f64,i64,f64
10241,1964-01-31,-0.051711,0.001913,0.001099,-0.000228,-0.000211,-0.000182,0.002112,null,null,0.002201,0.000498,-0.000395,0.000576,-0.001753,0.002188,0.000348,0.000984,null,-0.002202,-0.003593,-0.001116,0.000599,0.000283,-0.000399,-0.001638,0.001186,null,-0.000248,null,-0.001396,-0.001978,-0.001017,-0.001347,null,-0.00199,0.000315,-0.001022,0.001969,0.000657,null,null,null,null,0.000935,"""-0.002284802""",-0.000914,-0.001755,0.001257,"""0.0024067666""","""-0.003598671""",0.000109,0.000749,0.000799,-0.000136,null,671498.0,16378,0.0227
10575,1964-01-31,-0.001261,0.00189,null,null,null,null,-0.00161,null,null,-0.000054,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.000613,0.000954,null,null,null,0.000356,0.000154,-0.000903,null,-0.000184,null,null,null,null,-0.000184,"""-0.000688053""",0.000319,0.000618,-0.000944,"""-0.001791619""","""0.0025759719""",-0.00246,-0.000852,0.000799,-0.000407,null,693747.0,24342,0.0227
10591,1964-01-31,-0.025472,0.000136,null,null,null,null,-0.00161,null,null,0.000762,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.00026,0.000954,null,null,null,0.001167,0.002424,-0.000258,null,-0.001124,null,null,null,null,0.002301,"""-0.00043838""",-0.001768,0.000114,0.002253,"""0.0038523645""","""-0.002237635""",-0.002262,0.001865,0.000799,-0.000384,null,170681.0,2636,0.0227
10671,1964-01-31,0.102882,-0.001553,null,null,null,null,-0.00161,null,null,-0.000185,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-0.001856,-0.000029,null,null,null,-0.001922,-0.001424,-0.00125,null,-0.000313,null,null,null,null,0.000671,"""0.0017215851""",0.002027,0.001844,-0.000191,null,null,-0.000804,-0.002177,0.000799,0.001852,null,59626.0,4259,0.0227
11260,1964-01-31,-0.069066,0.002138,0.002674,-0.000293,-0.003917,0.002271,0.002112,null,null,0.001023,-0.003796,0.002283,0.001893,-0.00274,-0.004792,0.002944,0.00273,-0.009944,-0.005589,-0.004167,0.003144,0.002436,0.00266,-0.001501,-0.001845,0.002915,null,-0.001543,null,0.002299,0.001476,0.003618,0.003467,null,0.002438,0.002762,0.001022,-0.000021,-0.000473,null,null,null,null,-0.001492,"""0.0019248077""",0.002467,0.002052,-0.000469,null,null,0.000873,0.002169,0.000799,0.002228,null,1.405409e6,36152,0.0227
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
93312,2019-12-31,0.021081,0.000585,0.000519,-0.001169,0.000056,-0.001917,-0.001833,"""-0.000798722""","""0.0006518905""",0.001728,-0.00264,0.002224,-0.001466,0.000742,-0.002935,-0.001538,-0.001088,0.002254,0.001895,-0.00212,-0.001155,0.002226,0.000737,-0.001541,-0.00164,-0.000781,"""-0.002418546""",0.002477,null,-0.001888,0.000125,-0.001487,-0.002673,"""-0.000843749""",-0.000291,0.000225,0.001141,0.000429,0.001695,null,"""-0.001081183""","""-0.000639009""","""-0.00075545""",0.001787,"""0.0006800783""",0.000684,0.00117,0.00179,"""0.0030979187""","""-0.002245608""",0.001906,-0.000514,-0.001139,0.000105,null,1.563244e7,254600,0.0276
93330,2019-12-31,-0.025907,-0.000983,0.000117,-0.00195,null,null,-0.001166,"""0.0016129032""","""0.0006518905""",-0.001333,null,-0.000786,-0.001672,-0.001314,-0.002248,0.001026,-0.000012,-0.001718,-0.001714,0.00159,0.002025,0.000515,0.001568,-0.00072,0.000965,0.000034,"""-0.001092468""",0.00

In [31]:
stock_data_with_crsp_ff.null_count()

permno,date,re,size,value,prof,dur,valprof,fscore,debtiss,repurch,nissa,accruals,growth,aturnover,gmargins,divp,ep,cfp,noa,inv,invcap,igrowth,sgrowth,lev,roaa,roea,sp,gltnoa,divg,invaci,mom,indmom,valmom,valmomprof,shortint,mom12,momrev,lrrev,valuem,nissm,sue,roe,rome,roa,strev,ivol,betaarb,season,indrrev,indrrevlv,indmomrev,ciss,price,age,shvol,ipo,mkt_cap_monthly,SHROUT,Mkt-RF
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,163,7462,44282,40538,182182,128590,0,36445,36445,22282,135192,51005,35723,35775,175072,36248,81477,122621,111569,84616,100343,52121,35107,35018,43912,35595,175884,184584,458757,13363,3055,128163,132078,162753,15208,24489,79099,32991,15835,327564,89462,81336,112305,0,4681,47839,77277,3054,201725,201725,79197,0,0,12006,468906,5,0,0


In [32]:
# Format the `date` column as %m/%Y before saving to CSV
# Uses Polars' datetime strftime to produce 'MM/YYYY' strings
stock_data_with_crsp_ff = stock_data_with_crsp_ff.with_columns(
    pl.col("date").dt.strftime("%m/%Y").alias("date")
)
stock_data_with_crsp_ff.write_csv("data/stock_level_signals_updated.csv")

In [33]:
# ff_data.select(["date","Mkt-RF"]).write_csv("market_excess_return.csv")